# Trade

The purpose of this notebook is to review and visualize functionality in this app, namely around trade.

In [1]:
import os

os.chdir("../")

In [2]:
from trader.dao.dao import DAO

dao = DAO()

In [3]:
from trader.dao.markets import get_market_trade_goods_by_system
from trader.dao.waypoints import get_waypoints_by_system_symbol
from trader.roles.merchant.finder import generate_arbitrage_opportunities

system_symbol = "X1-BA28"
trade_goods = get_market_trade_goods_by_system(
    engine=dao.engine, system_symbol=system_symbol
)

waypoints = get_waypoints_by_system_symbol(
    engine=dao.engine, system_symbol=system_symbol
)
arbitrage_opportunities = generate_arbitrage_opportunities(
    market_trade_goods=trade_goods, waypoints=waypoints
)

In [4]:
from math import dist
from typing import Dict

from trader.roles.merchant.merchant import MAXIMUM_PERCENT_OF_ACCOUNT_PURCHASE
from trader.roles.merchant.finder import (
    ArbitrageOpportunity,
    find_most_profitable_trade_in_system,
)
from trader.roles.navigator.geometry import (
    generate_graph_from_waypoints_means_shift_clustering,
)

account_value = 200000
ship_speed = 30  # default frigate speed, apparently

waypoints = get_waypoints_by_system_symbol(
    engine=dao.engine, system_symbol=system_symbol
)

most_profitable_trade = find_most_profitable_trade_in_system(
    maximum_purchase_price=MAXIMUM_PERCENT_OF_ACCOUNT_PURCHASE * account_value,
    trade_goods=trade_goods,
    waypoints=waypoints,
    prefer_within_cluster=True,
)

distance = dist(
    [
        most_profitable_trade.purchase_waypoint.x,
        most_profitable_trade.purchase_waypoint.y,
    ],
    [
        most_profitable_trade.sell_waypoint.x,
        most_profitable_trade.sell_waypoint.y,
    ],
)
profit_distance_ratio = most_profitable_trade.profit / distance
print(
    f"Found most profitable trade of {most_profitable_trade.trade_good_symbol} (distance - {distance}): at {profit_distance_ratio} profit:distance ratio"
)
time_to_travel_distance = (distance * 2) / ship_speed  # should be in seconds
trips_per_hour = 3600 / time_to_travel_distance
profit_per_hour = trips_per_hour * most_profitable_trade.profit
print(f"At CRUISE for a frigate, this yields {profit_per_hour} of profit per hour")

Found most profitable trade of SHIP_PLATING (distance - 81.49233092751734): at 117.13985710496621 profit:distance ratio
At CRUISE for a frigate, this yields 6325552.283668175 of profit per hour
